In [18]:
import pandas as pd
import numpy as np

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from bs4 import BeautifulSoup
import re
import time

In [2]:
def stylebeerlinks(url):
    '''
    Funtion to find the all the beer links from style url
    ''' 
    #Set up dataframe to insert into
    resultdf = pd.DataFrame(columns=['link','beername','style'])
    
    soup = BeautifulSoup(requests.get(url).content,"lxml")
    style = soup.find('title').text.split('|')[0]
    beercount = int(soup.find('table').find('b').text.split(' ')[8][:-1])
    
    pages = beercount / 50
    pagenbr = 0
    
    while pagenbr < pages:
        urlext = '?sort=revsD&start={}'.format(pagenbr*50)
        linksoup = BeautifulSoup(requests.get(url+urlext).content,"lxml")

        table = linksoup.find('table')
        table = table.find_all('a')
        links = [str(x).split('"')[1] for x in table]
        beername = [x.text for x in table]
        datadict = {'link':links,'beername': beername}
        newdf = pd.DataFrame(datadict)
        newdf = newdf[newdf['link'].str.contains('/beer/profile/')]
        newdf = newdf[newdf['link'].str.count('/')==5]
        newdf['style'] = style

        resultdf = resultdf.append(newdf)
        
        pagenbr += 1
        time.sleep(1)

    return resultdf

In [19]:
def comments_pull(url):
    '''
    Funtion to pull all beer details and comments for a given beer advocate beer url
    '''
    #First we identify columns and a dataframe to input information into
    
    columns = ['beerid','breweryid','beername', 'brewery', 'style', 'abv', 'avgrating', 'pdev', 'reviewcount', 'ratingcount', 
           'rating', 'rdev', 'reviewer', 'date', 'look', 'smell', 'taste', 'feel', 'overall', 'text']
    resultdf = pd.DataFrame(columns=columns)
    
    #Next we pull the base data for the beer
    sessions = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    
    soup = BeautifulSoup(session.get(url).content,"lxml")
    
    breweryid = url.split('/')[-3]
    beerid = url.split('/')[-2]
    
    beername = soup.find(class_='titleBar').text.replace('\n','').split('|')[0]
    brewery = soup.find(class_='titleBar').text.replace('\n','').split('|')[1]
    style = soup.find(text='Style:').next_element.next_element.text
    abv = soup.find(text='Alcohol by volume (ABV):').next_element.replace('\t','').replace('\n','')
    avgrating = float(soup.find(class_='ba-ravg').text)
    pdev = soup.find(class_='ba-pdev').text.replace('\t','').replace('\n','')
    reviewcount = int(soup.find(class_='ba-reviews').text.replace(',',''))
    ratingcount = int(soup.find(class_='ba-ratings').text.replace(',',''))
    
    #next we go through the comments in 25 rating chunks until we can't anymore
    pagenbr = 0
    finish = True
    
    while finish:
        urlextention = '?view=beer&sort=&start={}'.format(pagenbr*25)
        newurl = url+urlextention
        commentsoup = BeautifulSoup(session.get(newurl).content,"lxml")
        reviewlist = commentsoup.find_all(class_='user-comment')
        if len(reviewlist)==0:
            break
        try:
            for reviewnbr in reviewlist:
                rating = float(reviewnbr.find(class_='BAscore_norm').text)
                rdev = reviewnbr.find(class_='rAvg_norm').next_element.next_element.next_element.text
                reviewer = reviewnbr.find_all(class_='username')[1].text
                date = reviewnbr.find_all(class_='username')[1].next_element.next_element.next_element.text
                try:
                    rate = reviewnbr.find(class_='muted').text
                    rate = rate.split('|')
                    rate = [x.split(':')[1].strip() for x in rate]
                    look = float(rate[0])
                    smell = float(rate[1])
                    taste = float(rate[2])
                    feel = float(rate[3])
                    overall = float(rate[4])
                except:
                    look = None
                    smell = None
                    taste = None
                    feel = None
                    overall = None

                textraw = reviewnbr.find_all('br')
                text = str()
                for b in textraw[2:-4]:
                    text += str(b.nextSibling.replace('\n',''))


                values = ([beerid,breweryid,beername, brewery, style, abv, avgrating, pdev, 
                           reviewcount, ratingcount, rating, rdev, reviewer, date, look, smell, taste, feel, overall, text])
                datadict = dict(zip(columns, values))
                newdf = pd.DataFrame(data=datadict,index=[0])
                resultdf = resultdf.append(newdf)
                

        except:
            pass
        
        pagenbr += 1 
        time.sleep(1)
        #print(newurl)
    return resultdf

In [10]:
url = 'https://www.beeradvocate.com/beer/style/'
soup = BeautifulSoup(requests.get(url).content,"lxml")
soup = soup.find('table')
soup = soup.find_all('a')
soup

[<a href="/beer/style/128/">American Amber / Red Ale</a>,
 <a href="/beer/style/19/">American Barleywine</a>,
 <a href="/beer/style/175/">American Black Ale</a>,
 <a href="/beer/style/99/">American Blonde Ale</a>,
 <a href="/beer/style/73/">American Brown Ale</a>,
 <a href="/beer/style/94/">American Dark Wheat Ale</a>,
 <a href="/beer/style/140/">American Double / Imperial IPA</a>,
 <a href="/beer/style/157/">American Double / Imperial Stout</a>,
 <a href="/beer/style/116/">American IPA</a>,
 <a href="/beer/style/97/">American Pale Ale (APA)</a>,
 <a href="/beer/style/93/">American Pale Wheat Ale</a>,
 <a href="/beer/style/159/">American Porter</a>,
 <a href="/beer/style/158/">American Stout</a>,
 <a href="/beer/style/78/">American Strong Ale</a>,
 <a href="/beer/style/171/">American Wild Ale</a>,
 <a href="/beer/style/130/">Black &amp; Tan</a>,
 <a href="/beer/style/163/">Chile Beer</a>,
 <a href="/beer/style/6/">Cream Ale</a>,
 <a href="/beer/style/72/">Pumpkin Ale</a>,
 <a href="/be

In [13]:
stylelinks = [str(x).split('"')[1] for x in soup]
stylelinks

['/beer/style/128/',
 '/beer/style/19/',
 '/beer/style/175/',
 '/beer/style/99/',
 '/beer/style/73/',
 '/beer/style/94/',
 '/beer/style/140/',
 '/beer/style/157/',
 '/beer/style/116/',
 '/beer/style/97/',
 '/beer/style/93/',
 '/beer/style/159/',
 '/beer/style/158/',
 '/beer/style/78/',
 '/beer/style/171/',
 '/beer/style/130/',
 '/beer/style/163/',
 '/beer/style/6/',
 '/beer/style/72/',
 '/beer/style/12/',
 '/beer/style/60/',
 '/beer/style/119/',
 '/beer/style/174/',
 '/beer/style/54/',
 '/beer/style/56/',
 '/beer/style/55/',
 '/beer/style/141/',
 '/beer/style/127/',
 '/beer/style/57/',
 '/beer/style/15/',
 '/beer/style/52/',
 '/beer/style/53/',
 '/beer/style/14/',
 '/beer/style/10/',
 '/beer/style/50/',
 '/beer/style/142/',
 '/beer/style/129/',
 '/beer/style/58/',
 '/beer/style/48/',
 '/beer/style/80/',
 '/beer/style/114/',
 '/beer/style/152/',
 '/beer/style/98/',
 '/beer/style/74/',
 '/beer/style/75/',
 '/beer/style/150/',
 '/beer/style/154/',
 '/beer/style/76/',
 '/beer/style/101/',


In [14]:
beersdf = pd.DataFrame(columns=['link','beername','style'])

url = 'https://www.beeradvocate.com'

for urlext in stylelinks:
    beersdf = beersdf.append(stylebeerlinks(url+urlext))
    beersdf.to_pickle('/home/michael/Documents/Projects/beerreview/data/beerlinksraw.pkl')

## 1.1 Pull Review Data

In [4]:
beersdf = pd.read_pickle('/home/michael/Documents/Projects/beerreview/data/beerlinks.pkl').reset_index(drop=True)

In [5]:
beersdf.shape

(211029, 3)

In [6]:
beersdf.head()

,beername,link,style
0,Fat Tire Amber Ale,/beer/profile/192/607/,American Amber / Red Ale
1,Nugget Nectar,/beer/profile/694/15881/,American Amber / Red Ale
2,Hop Head Red Ale,/beer/profile/2743/35732/,American Amber / Red Ale
3,Amber Ale,/beer/profile/287/794/,American Amber / Red Ale
4,Hopback Amber Ale,/beer/profile/694/6322/,American Amber / Red Ale


In [7]:
beersdf.groupby('style')[['link']].count()

,link
style,
Altbier,954
American Adjunct Lager,1166
American Amber / Red Ale,6533
American Amber / Red Lager,660
American Barleywine,1456
American Black Ale,2488
American Blonde Ale,4644
American Brown Ale,4349
American Dark Wheat Ale,199


In [8]:
beersdf['beercount'] = beersdf.groupby('style').cumcount()+1
beersdf.head()

,beername,link,style,beercount
0,Fat Tire Amber Ale,/beer/profile/192/607/,American Amber / Red Ale,1
1,Nugget Nectar,/beer/profile/694/15881/,American Amber / Red Ale,2
2,Hop Head Red Ale,/beer/profile/2743/35732/,American Amber / Red Ale,3
3,Amber Ale,/beer/profile/287/794/,American Amber / Red Ale,4
4,Hopback Amber Ale,/beer/profile/694/6322/,American Amber / Red Ale,5


In [9]:
columns = ['beerid','breweryid','beername', 'brewery', 'style', 'abv', 'avgrating', 'pdev', 'reviewcount', 'ratingcount', 
           'rating', 'rdev', 'reviewer', 'date', 'look', 'smell', 'taste', 'feel', 'overall', 'text']
reviewdf = pd.DataFrame(columns=columns)

In [10]:
group1 = beersdf[beersdf['beercount']==1]
group2 = beersdf[(beersdf['beercount']>=2) & (beersdf['beercount']<=10)]
group3 = beersdf[(beersdf['beercount']>=11) & (beersdf['beercount']<=25)]
group4 = beersdf[(beersdf['beercount']>=26) & (beersdf['beercount']<=50)]
group5 = beersdf[(beersdf['beercount']>=51) & (beersdf['beercount']<=100)]
group6 = beersdf[(beersdf['beercount']>=101) & (beersdf['beercount']<=200)]
group7 = beersdf[(beersdf['beercount']>=201) & (beersdf['beercount']<=400)]
group8 = beersdf[(beersdf['beercount']>=401) & (beersdf['beercount']<=700)]
group9 = beersdf[(beersdf['beercount']>=701) & (beersdf['beercount']<=1000)]

In [11]:
url = 'https://www.beeradvocate.com'

for index, rows in group1.iterrows():
    reviewdf = reviewdf.append(comments_pull(url+rows['link']))
    reviewdf.to_pickle('/home/michael/Documents/Projects/beerreview/data/beerreviews.pkl')

SSLError: HTTPSConnectionPool(host='www.beeradvocate.com', port=443): Max retries exceeded with url: /beer/profile/147/38470/?view=beer&sort=&start=5150 (Caused by SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')",),))

In [12]:
reviewdf= pd.read_pickle('/home/michael/Documents/Projects/beerreview/data/beerreviews.pkl')

In [13]:
reviewdf.shape

(16075, 20)

In [24]:
reviewdf.groupby('beername',as_index=False)[['beerid']].count()

,beername,beerid
0,Bigfoot Barleywine-Style Ale,7484
1,Fat Tire Amber Ale,8591


In [66]:
completed = reviewdf.groupby('beername',as_index=False)[['beerid']].count()
completed.columns = ['beername','reviewcount']
group1retry = pd.merge(group1, completed, how='left',left_on='beername',right_on='beername')
group1retry.head()

,beername,link,style,beercount,reviewcount
0,Fat Tire Amber Ale,/beer/profile/192/607/,American Amber / Red Ale,1,NaN
1,Bigfoot Barleywine-Style Ale,/beer/profile/140/2671/,American Barleywine,1,NaN
2,Sublimely Self-Righteous Black IPA,/beer/profile/147/38470/,American Black Ale,1,NaN
3,Summer Love,/beer/profile/345/59987/,American Blonde Ale,1,NaN
4,Indian Brown Ale,/beer/profile/64/1161/,American Brown Ale,1,NaN


In [65]:
completed.head()

,beername,reviewcount
0,Bigfoot Barleywine-Style Ale,7484
1,Fat Tire Amber Ale,8591
